# AST 245 Computational Astrophysics N-Body Project

### Libraries

```numba```: optimized running with decorater ```@jit(nopython=True)```, can also easily be parallelized with ```@jit(nopython=True, parallel=True)``` and using ```prange``` instead of ```range``` in the forloop -> careful: race condtion
```numpy```: allows vectorized calculations, improves performance as well
```matplotlib```: for plotting
```scipy.optimize```: from this library use the function ```curve_fit``` to fit a curve

### Data extraction

Kinda self explanatory

### Task 1: Step 1

Calculate a density profile and compare it with the density given from the Hernquist profile

To calculate the expected density for our data for a certain radius, use the calculation from Hernquist (1990):
$\rho(r) = \frac{M}{2 \pi} \frac{a}{r} \frac{1}{(r+a)^3}$, where *M* is the total mass and *a* is the scale length. We are given *M* and *r*, but *a* has to be fitted on the system with the data given and ```curve_fit```.

To calculate the particle density, we first have to divide the total data (spherical) in different shells. For each shell, the density can then be calculated, which then can be shown as a histogram, where the density for each shell = bin correlates with the number of particles in said shell.

To divide our data into equal bins and get the number of particles in each bin, use the function ```np.histogram``` on the absolute radius of each particle and the number of bins desired. This returns a tuple containing two values, firstly an array with the count of particles in each bin, and second an array with the edges of each bin.

This bin edges can be used to calculate the average radius for each bin with the formula ```(bin_edges[:-1] + bin_edges[1:]) / 2```, which works, as all are arrays and it also returns an array again. It takes for the first array all the elements but the last and for the second array all the elements but the first. This "shifted" arrays are then added together and each element is then divided by 2 to give the mid-radius of this shell.

The same procedure is used to get the inner and outer radii of each shell and then the volume of each shell is calculated based on the formula: $\frac{4}{3} * \pi * (R^3 - r^3)$ where *R* is the outer radius and *r* is the inner radius.

The particle density in each shell is then calculated based on the formula $\rho(r) = \frac{\text{particles in shell}}{\text{shell volume}}$. This is the **numerically calculated density**.

This numerically calculated density is the dependent parameter to be fitted with the Hernquist $\rho$ to the independent parameters of the radii of each bin and the total mass of the system. This results in the **analytically calculated density**.

Based on the numerically calculated and the analytically calculated density, the standard deviation and so the standard error of the numerically calculated density compared to the analytically calculated density can be calculated. For the standard deviation, ```np.std((model, obs), axis=0)``` can be used, which calculates the standard deviation with the formula $\sqrt{\frac{\sum_{i=1}^{N} (x_i - \bar{x})^2}{N}}$.

The standard error is then calculated as $\frac{\text{standard deviation}}{\text{Number of particles per bin}}$.

All of this is then plotted.

G = 1 -> Planck units
units of length 
units of mass 

to calculate back -> define the length in planck with the original SI units, then multiply with that value to get meters, etc.

### Task 1: Step 2

- Compute directly the forces for all the particles
- Start with assuming a softening of the order of the mean interparticle distance
- Repeat calculation with different orders of magnitude for the softening

The formula for the direct calculation of the force is $F(r) = a * m$, where $a = -G \sum_{j=1}^{N} \frac{m_j}{[(r_i - r_j)^2 + \epsilon^2]^{3/2}} (r_i - r_j)$. To get the total force in the end, calculate this as $F_{abs} = \sqrt{F_x^2 + F_y^2 + F_z^2}$. For the calculation, $G=1$. To improve performance, the outer loop of the calculation is parallelized, but the inner is not to avoid race conditions.

The softening parameter $\epsilon$ is added to the brute force calculation. To calculate the epsilon, that is used for a specific iteration of the direct force calculation:
- first the half mass radius is calculated. This is the radius, that contains half the mass of the system. For this, first the half mass is calculated as $\frac{\text{total mass of the system}{2}$. Then, from the sorted absolute radii of the particles, the index of the radius, that contains half the mass of the system is determined, which is then the half-mass radius.
- based on this radius, it is determined, which particles are contained within this radius. For this, a boolean mask is generated, that is a array of the same length as the (unsorted) absolute radii, that contains ```True``` if the particle's radius is within the half-mass radius, and ```False``` otherwise. This is then applied on the positions (x, y, z) of the particles.
- Based on this, the mean interparticle distance of the particles contained in the half-mass radius can be calculated, which is the sum of the distances for each particle to the other particles, divided by the number of interactions (which is the number of particles times the number of particles that influence each particle).
- As we should repeat the direct force calculation for several softening parameters, this mean interparticle distance is then used in a function, that also accepts an exponent for 10 and a step size. From this, a list of softening parameters is generated, with the lowest value $softening_{parameter} * (10^{-exp})$ and the highest value $softening_{parameter} * (10^{exp})$, with $stepsize$ steps.

This is then used in the function to generate a plot, for which the direct force calculated as often as there are values in the list of *softening parameters*. Plot only every 100th particle, otherwise the plot is useless.

**Interpretation of the plot**
When the softening is smaller, the magnitude of the force is higher. At smaller softenings, the effect of a small distance between two particles is more noticiable, as the force from one on the other is big and they would be pushed away from each other at a high speed. When the force is calculated without the softening, the force between two particles becomes large, when the particles approach each other closely, and F diverges, which is unphysical in the collisionless system as we have a smooth distribution.

To check the difference the softening makes, the forces calculated directly with different softenings are compared with the analytical solution calculated on the base of Newton's second shell theorem.

Newton's First Shell Theorem: *A body that is inside a spherical shell of matter experiences no net gravitational force from that shell.*

Newton's Second Shell Theorem: *The gravitational force on a body that lies outside a spherical shell of matter is the same as it would be if all the shell's matter were concentrated into a point at its centre.*

 According to Galactic Dynamics (2008) p. 62: *"From Newton's first and second theorems, if follows that the gravitational attraction of a spherical density distribution $\rho(r')$ on a unit mass at radius r is entirely determined by the mas interior to r"*: $F(r) = - \frac{GM(r)}{r^2} * ê_r$ where $M(r) = 4 \pi \int_0^r dr' r'^2 \rho(r')$
 
To solve this, I first split the volume of the particle into shells. For this I calculate the absolute radii of each particle and the maximum thereof. This maximal radius (which also determines the maximal sphere) is then used in ```np.linspace``` to divide it all in even spheres and also get the shell_thickness.

With ```in_shell = np.where((radii >= lower_bound) & (radii <= upper_bound))``` an array is created that contains an array for each shell, and in this shell are the indices of the particles in said shell. To return this as one array for other functions, the arrays are made the same length by padding with -1 as 0 would be an index and NAN makes a ton of problems.

Then the density $\rho$ for each shell is calculated. First, the mass of each shell is determined by summing the masses of all the particles in a shell. Then the volume is calculated by subtracting the volume of the inner sphere (determined by the inner shell radius) from the volume of the outer sphere (determined by the outer shell). The density of each shell is then calculated by dividing the mass of each shell by its volume.

Then the masses for the shells should be calculated, according to this upper integral, which is where I do not know how to proceed.

Finally, the forces are calculated and the absolute forces and the absolute radii are returned and can be added to the plot.